In [18]:
import pandas as pd
from yf_utils import random_slices, lookback_slices
from yf_utils import rank_perf, grp_tuples_sort_sum, top_set_sym_freq_cnt
from myUtils import pickle_load, pickle_dump

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 11)
pd.set_option("display.max_colwidth", 26)
# pd.set_option("display.width", 145)
pd.set_option("display.width", 280)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_close_clean = "df_close_clean"
fp_df_picks = "df_picks"
fp_l_days_to_drop = "l_days_to_drop"
# pickle file of dates missing from df_picks
fp_dates_missing_in_df_picks = 'dates_missing_in_df_picks'

In [19]:
df_picks = pickle_load(path_data_dump, fp_df_picks)
print(f'df_picks:\n{df_picks}')

df_picks:
    date_end_df_train  max_days_lookbacks     days_lookbacks sym_freq_15 sym_freq_14  ...                 sym_freq_6                 sym_freq_5                 sym_freq_4                 sym_freq_3 sym_freq_2
0          2023-11-13                 120      [30, 60, 120]          []          []  ...  ['ANF', 'CAH', 'LRN', ...  ['BTC-USD', 'EDU', 'US...      ['ALL', 'GPS', 'TGH']  ['CHE', 'CLS', 'DELL',...         []
1          2023-11-13                 120  [15, 30, 60, 120]          []          []  ...      ['ANF', 'CAH', 'NRG']  ['BTC-USD', 'EDU', 'US...  ['ALL', 'AMBC', 'CHE',...                         []         []
2          2023-11-10                 120      [30, 60, 120]          []          []  ...  ['ANF', 'CAH', 'LRN', ...        ['BTC-USD', 'ICPT']             ['EDU', 'GPS']  ['AGYS', 'ALL', 'CLS',...         []
3          2023-11-10                 120  [15, 30, 60, 120]          []          []  ...     ['ANF', 'CAH', 'USAC']  ['AGYS', 'BTC-USD', 'I...  ['AMBC', 

In [20]:
df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)
df_current = df_close_clean.copy()
print(f'df_current:\n{df_current}')

df_current:
                     A         AA        AAL        AAP        AAPL  ...       ZION        ZTO         ZTS       ZUMZ        ZWS
Date                                                                 ...                                                        
2017-11-29   65.633354  40.366665  48.002308  91.965721   40.065674  ...  41.108860  14.851051   68.809448  20.850000  23.835058
2017-11-30   66.158897  40.630943  49.210892  92.931862   40.625946  ...  41.158691  14.991067   69.462852  21.799999  24.145109
2017-12-01   65.633354  40.758194  47.758636  91.349251   40.436832  ...  41.598953  14.935060   69.472473  19.950001  23.747854
2017-12-04   63.292377  40.905018  48.665073  95.710609   40.141315  ...  42.977825  14.776375   68.242531  21.350000  24.145109
2017-12-05   62.900620  40.307930  48.216740  93.069862   40.103497  ...  42.263470  14.571016   68.079170  20.650000  24.154795
...                ...        ...        ...        ...         ...  ...        ...  

In [23]:
dates_missing_in_df_picks = pickle_load(path_data_dump, fp_dates_missing_in_df_picks)
print(f'dates_missing_in_df_picks, len({len(dates_missing_in_df_picks)}):\n{dates_missing_in_df_picks}')

dates_missing_in_df_picks, len(26):
['2023-07-17', '2023-07-19', '2023-07-24', '2023-07-28', '2023-07-31', '2023-08-16', '2023-08-21', '2023-08-22', '2023-08-23', '2023-08-24', '2023-08-28', '2023-08-29', '2023-08-30', '2023-08-31', '2023-09-06', '2023-09-07', '2023-09-19', '2023-09-26', '2023-09-27', '2023-10-03', '2023-10-09', '2023-10-10', '2023-10-11', '2023-10-13', '2023-10-16', '2023-10-18']


In [24]:
l_days_to_drop = pickle_load(path_data_dump, fp_l_days_to_drop)
print(f'l_days_to_drop, len({len(l_days_to_drop)}):\n{l_days_to_drop}') 

l_days_to_drop, len(26):
[85, 83, 80, 76, 75, 63, 60, 59, 58, 57, 55, 54, 53, 52, 49, 48, 40, 35, 34, 30, 26, 25, 24, 22, 21, 19]
